In [1]:
import torch
from torch import nn
import torchvision.datasets as datasets
import torchvision.models as models
import torch.optim as optim
from torch.utils.data import DataLoader

import sys
sys.path.append('./python_files/')
from model_trainer import ModelTrainer
from utils import debug_layers_dims
from data_loader import load_data
from nn_modules import View

In [2]:
torch.__version__

'0.4.1'

In [4]:
alexnet = models.alexnet()
cifar_train = datasets.CIFAR10('./data/cifar10', download=True)
cifar_test = datasets.CIFAR10('./data/cifar10', train=False, download=True)

Failed download. Trying https -> http instead. Downloading http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz to ./data/cifar10/cifar-10-python.tar.gz


KeyboardInterrupt: 

In [ ]:
train_input, train_target, test_input, test_target = load_data(cifar=True, normalize=True, flatten=False)
train_target = train_target.long()
test_target = test_target.long()

In [5]:
alexnet32 = nn.Sequential(
    nn.Conv2d(3, 32, kernel_size=(5, 5), stride=(2,2), padding=(1,1)),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1),
    nn.Conv2d(32, 192, kernel_size=(5,5), stride=(1, 1), padding=(2, 2)),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1),
    nn.Conv2d(192, 384, kernel_size=(3,3), stride=(1,1), padding=(1, 1)),
    nn.ReLU(),
    nn.Conv2d(384, 256, kernel_size=(3,3), stride=(1,1), padding=(1,1)),
    nn.ReLU(),
    nn.Conv2d(256, 256, kernel_size=(3,3), stride=(1,1), padding=(1,1)),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1),
    View([-1]),
    nn.Dropout(0.5),
    nn.Linear(256, 128),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(128, 128),
    nn.ReLU(),
    nn.Linear(128, 32)
    
)

In [8]:
debug_layers_dims(alexnet32, test_input.shape)

******************** Debugging layers sizes: ********************
input_shape= (_,3,32,32)
------------------
Conv2d(3, 32, kernel_size=(5, 5), stride=(2, 2), padding=(1, 1))
(_,3,32,32) --> (_,32,15,15)
------------------
ReLU()
(_,32,15,15) --> (_,32,15,15)
------------------
MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
(_,32,15,15) --> (_,32,7,7)
------------------
Conv2d(32, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
(_,32,7,7) --> (_,192,7,7)
------------------
ReLU()
(_,192,7,7) --> (_,192,7,7)
------------------
MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
(_,192,7,7) --> (_,192,3,3)
------------------
Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
(_,192,3,3) --> (_,384,3,3)
------------------
ReLU()
(_,384,3,3) --> (_,384,3,3)
------------------
Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
(_,384,3,3) --> (_,256,3,3)
------------------
ReLU()
(_,256,3,3) --> (_,

tensor([[-0.0836, -0.0107, -0.0777, -0.1000, -0.0559,  0.0176, -0.0026,  0.0761,
          0.0652, -0.0187,  0.0143,  0.0380,  0.0945, -0.0476, -0.0799,  0.0629,
          0.0503,  0.0517, -0.0534, -0.0676,  0.0780, -0.0047,  0.0567, -0.0494,
         -0.0117, -0.0395, -0.0340,  0.0361,  0.0128,  0.0071, -0.0409,  0.0434],
        [-0.0883, -0.0086, -0.0816, -0.0930, -0.0430,  0.0243, -0.0080,  0.0790,
          0.0618, -0.0294,  0.0022,  0.0265,  0.1059, -0.0484, -0.0787,  0.0533,
          0.0461,  0.0522, -0.0639, -0.0722,  0.0866, -0.0079,  0.0475, -0.0545,
         -0.0179, -0.0375, -0.0409,  0.0361,  0.0015,  0.0103, -0.0274,  0.0465]],
       grad_fn=<ThAddmmBackward>)

## 1) Alexnet with disjoint trainset and testset

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adamax(alexnet.parameters(), lr=0.001)

* Using CIFAR
Files already downloaded and verified
Files already downloaded and verified
** Reduce the data-set (use --full for the full thing)
** Use 1000 train and 1000 test samples


In [10]:
target.reshape(-1, 1)

NameError: name 'target' is not defined

In [9]:
crit_fun = lambda input_, target : (input_, target.reshape(-1, 1))
y_hat_fun = lambda y: y.max(1)

mt = ModelTrainer(alexnet32, criterion, optimizer, y_hat_fun, crit_fun)
hist = mt.fit((train_input, train_target), (test_input, test_target), epochs=500, batch_size=250, verbose=10)
mt.plot_training("Learning curves")

RuntimeError: multi-target not supported at /Users/soumith/miniconda2/conda-bld/pytorch_1532623076075/work/aten/src/THNN/generic/ClassNLLCriterion.c:21